# Aula extra - Como analisar os dados de rentabilidade?

### Desafio:

* Construir um código que faça um ranking dos melhores fundos em três períodos de rentabilidade diferentes. Iremos selecionar os 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.

* Essa aula será mais curta. 

### Passo a passo:

   **Passo 1** - Puxar as tabelas igual na aula 2.
   
   **Passo 2** - Escolher quais períodos de rentabilidade iremos analisar.
   
   **Passo 3** - Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

   **Passo 4** - Transformar as rentabilidades em números decimais, ao invés de string. 
   
   **Passo 5** - Juntar as tabelas, igual na aula 2. 
   
   **Passo 6** - Filtrar os ETFs alavancados. 
   
   **Passo 7** - Fazer os rankings de rentabilidade por período e soma-los. 
   
   **Passo 8** - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

# Passo 1: Puxar as tabelas igual na aula 2.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [ ]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

time.sleep(5)

botao_100 = driver.find_element("xpath", '''html/body/div[5]/section/div/div[3]/section/div
                                                /div/div/div/div[2]/
                                section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao_100)

numero_paginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/
section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)

botao_aba = driver.find_element("xpath", ''' /html/body/div[5]/section/div/div[3]/section/div/div/div/div/
div[2]/section[2]/div[2]/ul/li[2]/span''')

driver.execute_script("arguments[0].click();", botao_aba)

#voltando a 1 página

for pagina in range(0, numero_paginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)
    
    
lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)

driver.quit()

In [ ]:
base_de_dados_completa = base_de_dados_completa.set_index('Ticker')
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")

In [ ]:
base_de_dados_performance

# Passo 2: Escolher quais períodos de rentabilidade iremos analisar.

No nosso caso, iremos pegar as janelas de 3, 5 e 10 anos. 

In [ ]:
base_de_dados_performance = base_de_dados_performance[['3 Years', '5 Years', '10 Years']]

base_de_dados_performance

# Passo 3: Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

In [55]:
base_de_dados_performance = base_de_dados_performance.replace("--", pd.NA)

base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,7.75%,8.99%,12.39%
IVV,7.75%,9.03%,12.44%
VTI,7.24%,8.49%,12.02%
VOO,7.73%,9.02%,12.44%
QQQ,8.84%,11.98%,16.55%
...,...,...,...
SZK,-26.41%,-20.50%,-22.95%
LTL,-6.32%,-2.87%,2.44%
LD,2.50%,-5.95%,-1.99%


In [56]:
base_de_dados_performance = base_de_dados_performance.dropna()

base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,7.75%,8.99%,12.39%
IVV,7.75%,9.03%,12.44%
VTI,7.24%,8.49%,12.02%
VOO,7.73%,9.02%,12.44%
QQQ,8.84%,11.98%,16.55%
...,...,...,...
SZK,-26.41%,-20.50%,-22.95%
LTL,-6.32%,-2.87%,2.44%
LD,2.50%,-5.95%,-1.99%


# Passo 4: Transformar as rentabilidades em números decimais

In [57]:
base_de_dados_performance['3 Years'] = (base_de_dados_performance['3 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['5 Years'] = (base_de_dados_performance['5 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['10 Years'] = (base_de_dados_performance['10 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,0.0775,0.0899,0.1239
IVV,0.0775,0.0903,0.1244
VTI,0.0724,0.0849,0.1202
VOO,0.0773,0.0902,0.1244
QQQ,0.0884,0.1198,0.1655
...,...,...,...
SZK,-0.2641,-0.2050,-0.2295
LTL,-0.0632,-0.0287,0.0244
LD,0.0250,-0.0595,-0.0199


# Passo 5: Juntar as tabelas, igual na aula 2.

In [59]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance, how = "inner")

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.93B,0.0775,0.0899,0.1239
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$298.80B,0.0775,0.0903,0.1244
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$276.32B,0.0724,0.0849,0.1202
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$275.55B,0.0773,0.0902,0.1244
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$148.30B,0.0884,0.1198,0.1655
...,...,...,...,...,...,...,...,...
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.47M,-0.2641,-0.2050,-0.2295
LTL,ProShares Ultra Telecommunications,Leveraged Equity: U.S. Telecoms,ProShares,0.95%,$1.34M,-0.0632,-0.0287,0.0244
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.20M,0.0250,-0.0595,-0.0199


# Passo 6: Filtrar os ETFs alavancados.

In [60]:
base_de_dados_final = base_de_dados_final[~base_de_dados_final['Segment'].str.contains("Leveraged")]

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.93B,0.0775,0.0899,0.1239
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$298.80B,0.0775,0.0903,0.1244
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$276.32B,0.0724,0.0849,0.1202
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$275.55B,0.0773,0.0902,0.1244
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$148.30B,0.0884,0.1198,0.1655
...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.89M,-0.2775,-0.2535,-0.2925
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.47M,-0.2641,-0.2050,-0.2295
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.20M,0.0250,-0.0595,-0.0199


# Passo 7: Fazer os rankings de rentabilidade por período e soma-los. 

In [61]:
base_de_dados_final['rank_3_anos'] = base_de_dados_final['3 Years'].rank(ascending = False)
base_de_dados_final['rank_5_anos'] = base_de_dados_final['5 Years'].rank(ascending = False)
base_de_dados_final['rank_10_anos'] = base_de_dados_final['10 Years'].rank(ascending = False)

base_de_dados_final

C:\Users\lsiqu\AppData\Local\Temp\ipykernel_12136\2550834387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_final['rank_3_anos'] = base_de_dados_final['3 Years'].rank(ascending = False)
C:\Users\lsiqu\AppData\Local\Temp\ipykernel_12136\2550834387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_final['rank_5_anos'] = base_de_dados_final['5 Years'].rank(ascending = False)
C:\Users\lsiqu\AppData\Local\Temp\ipykernel_12136\2550834387.py:3: SettingWithCopyWarning: 
A value is

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos
Ticker,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.93B,0.0775,0.0899,0.1239,202.0,108.0,78.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$298.80B,0.0775,0.0903,0.1244,202.0,101.5,76.0
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$276.32B,0.0724,0.0849,0.1202,235.0,137.0,104.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$275.55B,0.0773,0.0902,0.1244,204.5,104.0,76.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$148.30B,0.0884,0.1198,0.1655,151.0,22.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.89M,-0.2775,-0.2535,-0.2925,805.0,811.0,818.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.47M,-0.2641,-0.2050,-0.2295,804.0,801.0,807.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.20M,0.0250,-0.0595,-0.0199,451.0,760.0,703.5


In [62]:
base_de_dados_final['rank_final'] =  (base_de_dados_final['rank_3_anos'] + 
                                      base_de_dados_final['rank_5_anos'] + 
                                      base_de_dados_final['rank_10_anos'])

base_de_dados_final

C:\Users\lsiqu\AppData\Local\Temp\ipykernel_12136\1151530422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_final['rank_final'] =  (base_de_dados_final['rank_3_anos'] +


,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.93B,0.0775,0.0899,0.1239,202.0,108.0,78.5,388.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$298.80B,0.0775,0.0903,0.1244,202.0,101.5,76.0,379.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$276.32B,0.0724,0.0849,0.1202,235.0,137.0,104.5,476.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$275.55B,0.0773,0.0902,0.1244,204.5,104.0,76.0,384.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$148.30B,0.0884,0.1198,0.1655,151.0,22.0,13.0,186.0
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.89M,-0.2775,-0.2535,-0.2925,805.0,811.0,818.0,2434.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.47M,-0.2641,-0.2050,-0.2295,804.0,801.0,807.0,2412.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.20M,0.0250,-0.0595,-0.0199,451.0,760.0,703.5,1914.5


# Passo 8: Gran finale - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

In [63]:
melhores_etfs = base_de_dados_final.sort_values(by = "rank_final")

melhores_etfs

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.59B,0.3308,0.2532,0.1735,2.0,1.0,8.0,11.0
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,Equity: U.S. Energy,First Trust,0.58%,$1.78B,0.2418,0.1953,0.1860,13.0,3.0,5.0,21.0
XSD,SPDR S&P Semiconductor ETF,Equity: U.S. Semiconductors,State Street Global Advisors,0.35%,$1.20B,0.1900,0.2060,0.2364,29.5,2.0,1.0,32.5
SMH,VanEck Semiconductor ETF,Equity: Global Semiconductors,VanEck,0.35%,$6.87B,0.1677,0.1766,0.2242,37.0,5.0,4.0,46.0
PSI,Invesco Dynamic Semiconductors ETF,Equity: U.S. Semiconductors,Invesco,0.56%,$501.86M,0.1612,0.1541,0.2286,43.0,9.0,2.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...
FAZ,Direxion Daily Financial Bear 3X Shares,Inverse Equity: U.S. Financials,Direxion,1.01%,$173.74M,-0.5603,-0.4670,-0.4593,832.0,832.0,831.0,2495.0
SSG,ProShares UltraShort Semiconductors,Inverse Equity: U.S. Semiconductors,ProShares,0.95%,$8.01M,-0.5494,-0.4862,-0.4707,831.0,833.0,833.0,2497.0
SQQQ,ProShares UltraPro Short QQQ,Inverse Equity: U.S. - Large Cap,ProShares,0.95%,$4.51B,-0.5451,-0.5159,-0.5103,830.0,834.0,834.0,2498.0


In [64]:
melhores_etfs.head(10)

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.59B,0.3308,0.2532,0.1735,2.0,1.0,8.0,11.0
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,Equity: U.S. Energy,First Trust,0.58%,$1.78B,0.2418,0.1953,0.1860,13.0,3.0,5.0,21.0
XSD,SPDR S&P Semiconductor ETF,Equity: U.S. Semiconductors,State Street Global Advisors,0.35%,$1.20B,0.1900,0.2060,0.2364,29.5,2.0,1.0,32.5
SMH,VanEck Semiconductor ETF,Equity: Global Semiconductors,VanEck,0.35%,$6.87B,0.1677,0.1766,0.2242,37.0,5.0,4.0,46.0
PSI,Invesco Dynamic Semiconductors ETF,Equity: U.S. Semiconductors,Invesco,0.56%,$501.86M,0.1612,0.1541,0.2286,43.0,9.0,2.0,54.0
SOXX,iShares Semiconductor ETF,Equity: U.S. Semiconductors,Blackrock,0.35%,$6.58B,0.1513,0.1704,0.2284,48.0,6.0,3.0,57.0
ICLN,iShares Global Clean Energy ETF,Equity: Global Renewable Energy,Blackrock,0.40%,$5.30B,0.1913,0.1798,0.1281,28.0,4.0,62.0,94.0
PTF,Invesco DWA Technology Momentum ETF,Equity: U.S. Information Technology,Invesco,0.60%,$207.64M,0.1271,0.1614,0.1584,74.0,7.0,17.0,98.0
XLK,Technology Select Sector SPDR Fund,Equity: U.S. Information Technology,State Street Global Advisors,0.10%,$40.22B,0.1166,0.1544,0.1787,84.0,8.0,7.0,99.0
